In [68]:
import pandas as pd
import re
import numpy as np
import os
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [69]:
df=pd.read_json("data_3000.json")
#df.head(20)
tittle_1=df['title'][0]
tittle_=tittle_1
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2868 entries, 0 to 2867
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    2868 non-null   object
 1   details  2868 non-null   object
dtypes: object(2)
memory usage: 44.9+ KB


In [70]:
#title_0=df['title'][0]
df['title']=df['title'].astype('str') 
df['details']=df['details'].astype('str') #date
#df['date']=df['date'].astype('str')

def clean_data(text):
    for ch in ['','\ufeff','\u200b','\""','.',',','%','\\','`','«', '_','{','}','[',']','(',')','>','#','+','-','!','$','\'','/','»',':','“','”','xa0','""','1','2','3','4','5','6','7','8','9','0','?','!','’']:
        if ch in text:
            text = text.replace(ch,"")         
    return text




def clean_scraped_data():
    for j in range(0,2867):
        df['title'][j]=clean_data(df['title'][j])
        df['details'][j]=clean_data(df['details'][j])
        #df['date'][j]=clean_data(df['date'][j])
         
     
    return df                     


df=clean_scraped_data()
df
    

,title,details
0,Covid morts la baisse se poursuit dans les s...,Le nombre de malades du Covid dans les service...
1,Covid un essai lancé en France sur linterchan...,Une dose de Moderna puis une dose de Pfizer Bi...
2,Covid malgré son retard initial la France fai...,Après des débuts difficiles la campagne vaccin...
3,Covid vaccinodrome cherche vaccinateurs le jo...,Après une campagne de recrutement organisée pa...
4,Covid la baisse des hospitalisations se poursuit,Le nombre de malades du Covid à lhôpital servi...
...,...,...
2863,"Vaccin en France ""il est très difficile de sa...",Alors que la campagne française de vaccination...
2864,Succès inattendu pour les stations de ski malg...,Neige abondante soleil Lors des vacances de No...
2865,Covid comment les Français concernés par le c...,Pas moins de six millions de Français de lest ...
2866,Covid vaccination à grande échelle à Pékin av...,Plus de Pékinois ont été vaccinés contre le c...


In [71]:
data=  pd.DataFrame()
data['text'] = df['title'] + " " + df['details']
#df = df.drop([ 'date'],axis=1)

In [72]:
#text="" # data sous forme text (one string)
#with open('data_cleaned_.txt', 'w') as f:
  #  for i in range(0,2867):
   #     text=text+str(data['text'][i])
    #    f.write(str(data['text'][i]))       

In [73]:
data['text']

0       Covid   morts la baisse se poursuit dans les s...
1       Covid  un essai lancé en France sur linterchan...
2       Covid  malgré son retard initial la France fai...
3       Covid  vaccinodrome cherche vaccinateurs le jo...
4       Covid  la baisse des hospitalisations se pours...
                              ...                        
2863    Vaccin  en France "il est très difficile de sa...
2864    Succès inattendu pour les stations de ski malg...
2865    Covid  comment les Français concernés par le c...
2866    Covid  vaccination à grande échelle à Pékin av...
2867    Covid-19 : le vaccin de Moderna va être autori...
Name: text, Length: 2868, dtype: object

In [74]:
import random

foo = [0,1]
#print(random.choice(foo))

In [75]:
data['label']=0

In [76]:
for i in range(0,2867):
    data['label'][i]=random.choice(foo)


<ipython-input-76-027596656ef6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'][i]=random.choice(foo)


In [77]:
no_of_fakes = data.loc[data['label'] == 1].count()[0]
no_of_trues = data.loc[data['label'] == 0].count()[0]
print(no_of_fakes)
print(no_of_trues)

1404
1464


### Data pre-processing

In [78]:
# Step - a : Remove blank rows 
data['text'].dropna(inplace=True)

In [79]:
# Step - b : Change all the text to lower case.  
data['text'] = [entry.lower() for entry in data['text']]

In [81]:
for i in range(0,2867):
    mytxt=""
    mytxt=str(data['text'][i])
print(data['text'])

0       [covid, morts, la, baisse, se, poursuit, dans,...
1       [covid, un, essai, lancé, en, france, sur, lin...
2       [covid, malgré, son, retard, initial, la, fran...
3       [covid, vaccinodrome, cherche, vaccinateurs, l...
4       [covid, la, baisse, des, hospitalisations, se,...
                              ...                        
2863    [vaccin, en, france, ``, il, est, très, diffic...
2864    [succès, inattendu, pour, les, stations, de, s...
2865    [covid, comment, les, français, concernés, par...
2866    [covid, vaccination, à, grande, échelle, à, pé...
2867    [covid-19, :, le, vaccin, de, moderna, va, êtr...
Name: text, Length: 2868, dtype: object


In [80]:
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
data['text']= [word_tokenize(entry) for entry in data['text']]

In [82]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop


final_stopwords_list = list(fr_stop) 


In [83]:
import spacy
from spacy.lang.fr.examples import sentences 
import fr_core_news_md
from spacy.lang.en import English     
##-----------
nlp = spacy.load('fr_core_news_md')     
##-----------
for index,entry in enumerate(data['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V)  
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        # 
        if word not in final_stopwords_list and word.isalpha():
            #word_Final = word_Lemmatized.lemmatize(word)
            word_Final = word
            #steaming
            doc = nlp(str(word_Final)) 
            for token in doc:
                word_Final =  token.lemma_
            
            #word_Final=stemming(word_Final)
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    data.loc[index,'text_final'] = str(Final_words)

In [84]:
data['text_final']# dataframe removed stopword , applied  steaming !

0       ['covid', 'mort', 'baisse', 'poursuivre', 'ser...
1       ['covid', 'essai', 'lancer', 'france', 'linter...
2       ['covid', 'malgré', 'retard', 'initial', 'fran...
3       ['covid', 'vaccinodrome', 'cherche', 'vaccinat...
4       ['covid', 'baisse', 'hospitalisation', 'poursu...
                              ...                        
2863    ['vaccin', 'france', 'très', 'difficile', 'sav...
2864    ['succès', 'inattendu', 'station', 'ski', 'mal...
2865    ['covid', 'français', 'concerner', 'couvrefeu'...
2866    ['covid', 'vaccination', 'grand', 'échell', 'p...
2867    ['vaccin', 'moderna', 'autoriser', 'heure', 'j...
Name: text_final, Length: 2868, dtype: object

In [96]:
data

,text,label,text_final
0,"[covid, morts, la, baisse, se, poursuit, dans,...",1,"['covid', 'mort', 'baisse', 'poursuivre', 'ser..."
1,"[covid, un, essai, lancé, en, france, sur, lin...",1,"['covid', 'essai', 'lancer', 'france', 'linter..."
2,"[covid, malgré, son, retard, initial, la, fran...",1,"['covid', 'malgré', 'retard', 'initial', 'fran..."
3,"[covid, vaccinodrome, cherche, vaccinateurs, l...",0,"['covid', 'vaccinodrome', 'cherche', 'vaccinat..."
4,"[covid, la, baisse, des, hospitalisations, se,...",1,"['covid', 'baisse', 'hospitalisation', 'poursu..."
...,...,...,...
2863,"[vaccin, en, france, ``, il, est, très, diffic...",0,"['vaccin', 'france', 'très', 'difficile', 'sav..."
2864,"[succès, inattendu, pour, les, stations, de, s...",0,"['succès', 'inattendu', 'station', 'ski', 'mal..."
2865,"[covid, comment, les, français, concernés, par...",1,"['covid', 'français', 'concerner', 'couvrefeu'..."
2866,"[covid, vaccination, à, grande, échelle, à, pé...",0,"['covid', 'vaccination', 'grand', 'échell', 'p..."


# ---------------------------------------------------------------------------------

In [161]:
data['text_final'][0]

"['covid', 'mort', 'baisse', 'poursuivre', 'service', 'réanimation', 'nombre', 'malade', 'covid', 'service', 'réanimation', 'poursuivre', 'baisse', 'mardi', 'total', 'hospitalisation', 'mois', 'chiffre', 'santé', 'public', 'france', 'service', 'soin', 'critique', 'compter', 'patient', 'contre', 'veille']"

In [163]:
out = df.to_json(orient='records')[1:-1].replace('},{', '} {')
with open('file_name.txt', 'w') as f:
    f.write(out)

### Prepare Train and Test Data sets

In [86]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data['text_final'],data['label'],test_size=0.2,random_state=42)

Train_X.shape, Test_X.shape, Train_Y.shape, Test_Y.shape

((2294,), (574,), (2294,), (574,))

In [87]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

 

In [88]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [89]:
#pd.DataFrame(data=Train_X_Tfidf.toarray(), columns=Tfidf_vect.vocabulary_)
print(Tfidf_vect.vocabulary_)

{'covid': 925, 'mort': 2981, 'baisse': 323, 'poursuivre': 3454, 'service': 4206, 'réanimation': 3976, 'nombre': 3100, 'malade': 2790, 'mardi': 2817, 'total': 4587, 'hospitalisation': 2072, 'mois': 2957, 'chiffre': 613, 'santé': 4125, 'public': 3638, 'france': 1862, 'soin': 4276, 'critique': 942, 'compter': 760, 'patient': 3288, 'contre': 874, 'veille': 4751, 'essai': 1643, 'lancer': 2396, 'vaccin': 4718, 'arn': 212, 'messager': 2890, 'dose': 1321, 'moderna': 2949, 'pfizer': 3348, 'biontech': 393, 'linvers': 2622, 'mai': 2776, 'cours': 916, 'regarder': 3801, 'très': 4665, 'proche': 3504, 'céter': 989, 'cas': 541, 'vaccination': 4722, 'simplifier': 4236, 'faciliter': 1730, 'malgré': 2794, 'retard': 3889, 'initial': 2178, 'partie': 3268, 'bon': 414, 'élève': 4944, 'début': 1379, 'difficile': 1198, 'campagne': 511, 'vaccinale': 4720, 'français': 1871, 'rattraper': 3742, 'voisin': 4833, 'européen': 1665, 'million': 2922, 'recevoir': 3757, 'injection': 2182, 'population': 3428, 'louvertur': 

In [90]:
print(Train_X_Tfidf)

  (0, 4369)	0.2684725683709742
  (0, 4271)	0.4467818163118111
  (0, 4270)	0.2752994322700048
  (0, 3359)	0.2532902983337102
  (0, 2981)	0.12926935646812251
  (0, 2961)	0.19220338802648398
  (0, 2894)	0.14947325010551163
  (0, 2856)	0.15855065045583674
  (0, 2790)	0.15822074448770806
  (0, 2292)	0.12261977297878596
  (0, 2286)	0.2627005371881062
  (0, 2248)	0.14195931949558302
  (0, 1952)	0.29442683042802387
  (0, 1875)	0.2532902983337102
  (0, 1862)	0.08884703838697798
  (0, 1693)	0.19302636808137755
  (0, 1164)	0.1283706552849778
  (0, 941)	0.25615163689085435
  (0, 925)	0.11617938945809977
  (0, 286)	0.25770057635118854
  (1, 4971)	0.11375050927838094
  (1, 4849)	0.09422649160696421
  (1, 4835)	0.13331335505254868
  (1, 4746)	0.32043362126564834
  (1, 4475)	0.14472804351467644
  :	:
  (2292, 291)	0.12951842643749328
  (2292, 15)	0.32384526994921947
  (2293, 4872)	0.09837248838408309
  (2293, 4849)	0.0997705514610661
  (2293, 4593)	0.31060442138058836
  (2293, 4404)	0.1300367523163618

### Naive Bayes Classifier Algorithm

In [91]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy

print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*1)
predictions_NB

Naive Bayes Accuracy Score ->  0.5331010452961672


array([1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,

## ----------------------------

### SVM Algorithm Classifier

In [128]:
# fit the training dataset on the classifier
SVM = svm.SVC(C=2.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*1)

SVM Accuracy Score ->  0.5139372822299652


## -----------------------

In [94]:
from sklearn.naive_bayes import GaussianNB
from time import time
t0 = time()
model = GaussianNB()
model.fit(Train_X_Tfidf.toarray(), Train_Y)
print(f"\nTraining time: {round(time()-t0, 3)}s")
t0 = time()
score_train = model.score(Train_X_Tfidf.toarray(), Train_Y)
print(f"Prediction time (train): {round(time()-t0, 3)}s")
t0 = time()
score_test = model.score(Train_X_Tfidf.toarray(), Train_Y)
print(f"Prediction time (test): {round(time()-t0, 3)}s")
print("\nTrain set score:", score_train)
print("Test set score:", score_test)


Training time: 0.213s
Prediction time (train): 0.318s
Prediction time (test): 0.307s

Train set score: 0.9110723626852659
Test set score: 0.9110723626852659


In [173]:
def enter_news():
    news=input("Enter news a tester :  ")
    return news
news=  enter_news()
##----------------------------------------------------------
data3= pd.DataFrame() 

def clean_data(text):
     
    for ch in ['·','\ufeff','\u200b','\""','.',',','%','\\','`','«', '_','{','}','[',']','(',')','>','#','+','-','!','$','\'','/','»',':','“','”','xa0','""','1','2','3','4','5','6','7','8','9','0','?','!','’']:
        if ch in text:
            text = text.replace(ch,"")
           
    return text

 
df2=clean_data(news)
data3.loc[0,'text'] = str(df2)
data3['text']  
data3
##----------------------------------------------------------
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
data3['text'] = [entry.lower() for entry in data3['text']]

# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
data3['text']= [word_tokenize(entry) for entry in data3['text']]


data3['text']
##----------------------------------------------------------
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

final_stopwords_list = list(fr_stop) 

##----------------------------------------------------
import spacy
from spacy.lang.fr.examples import sentences 
import fr_core_news_md
from spacy.lang.en import English     
##-----------
nlp = spacy.load('fr_core_news_md')     
##-----------
for index,entry in enumerate(data3['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        # 
        if word not in final_stopwords_list and word.isalpha():
            #word_Final = word_Lemmatized.lemmatize(word)
            word_Final = word
            #steaming
            doc = nlp(str(word_Final)) 
            for token in doc:
                word_Final =  token.lemma_
            
            #word_Final=stemming(word_Final)
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    data3.loc[index,'text_final'] = str(Final_words)
##---------------------------------------------------------------------------

 
 
data3['text_final'][0]
news_p= Tfidf_vect.transform(data3['text_final'])
predictions_news_p = Naive.predict(news_p)

resultat = "****  Naive Bayes  :    "
#print(" **************************  Naive Bayes  : ")
if predictions_news_p==1:
    
    resultat=resultat+"Real"
else :
    resultat=resultat+"Fake"
#print(" **************************  SVM : ")
predictions_nes = SVM.predict(news_p)

resultat=resultat+ " ||  ********  SVM : "
if predictions_nes==1:
    
    resultat=resultat+"Real"
else :
    resultat=resultat+"Fake"

print("\n\n")
print(resultat)    

Enter news a tester :  Covid  231.800 doses administrées, 109 nouveaux cas ce lundi 14 juin 2021 COMPENSATION : LA CHARGE EXPLOSE À FIN MAI 2021 TAN-TAN: 35 MIGRANTS SUBSAHARIENS SECOURUS  Plus de 2.300 personnes ont reçu la première dose. Le cumul atteint 9,36 millions de personnes. En parallèle, 229.565 personnes ont reçu la deuxième dose, soit 7,32 millions au total. 18 nouveaux cas sévères ont été enregistrés ce lundi 14 juin.  ----------------------------------------------  Enter news a tester :  Une star de la chanson francophone vient compléter ce jury: Mylène Farmer. La native du Québec, interprète de tubes comme "Sans contrefaçon" ou "Désenchantée", est "indissociable du monde de l'image et du cinéma qui la fascine depuis toujours", notamment au travers de ses clips, souligne le festival.   --------------------------------------------  la tristesse ou une humeur fluctuante;l'irritabilité; l'anxiété;des troubles du sommeil (insomnie ou hypersomnie);une perte ou un gain de poids